In [ ]:
import pandas as pd
# Ranking module also shared as a gist: https://gist.github.com/ceroper/58675b2ac2c73b66f24f63c32e837af2
from ranking import *
from utils import *
from sklearn import svm, linear_model
from sklearn.model_selection import GroupKFold
from sklearn.metrics import ndcg_score
from sklearn.datasets import load_svmlight_file
from sklearn.exceptions import ConvergenceWarning

In [15]:
# Downloaded Learn to Rank dataset here: https://huggingface.co/datasets/YahooResearch/Yahoo-Learning-to-Rank-Challenge
# Found "convert" function source here: https://colab.research.google.com/github/finardi/tutos/blob/master/learning_to_rank.ipynb/#scrollTo=vmpCW_oepNCZ
## Note that we have convert the original raw data into a pure libsvm format.
## For more details, pls refer to: https://github.com/guolinke/boosting_tree_benchmarks/tree/master/data

def convert(input_filename, out_data_filename, out_query_filename, out_query_filename2):
	input = open(input_filename,"r")
	output_feature = open(out_data_filename,"w")
	output_query = open(out_query_filename,"w")
	output_query2 = open(out_query_filename2,"w")
	cur_cnt = 0
	cur_doc_cnt = 0
	last_qid = -1
	while True:
		line = input.readline()
		if not line:
			break
		tokens = line.split(' ')
		tokens[-1] = tokens[-1].strip()
		label = tokens[0]
		qid = int(tokens[1].split(':')[1])
		if qid != last_qid:
			if cur_doc_cnt > 0:
				output_query.write(str(cur_doc_cnt) + '\n')
				output_query2.write(str(cur_doc_cnt) + '\n')
				cur_cnt += 1
			cur_doc_cnt = 0
			last_qid = qid
		cur_doc_cnt += 1
		output_feature.write(label+' ')
		output_feature.write(' '.join(tokens[2:]) + '\n')
	output_query.write(str(cur_doc_cnt) + '\n')
	output_query2.write(str(cur_doc_cnt) + '\n')
	
	input.close()
	output_query.close()
	output_feature.close()
	output_query2.close()

convert("Learning to Rank Challenge/ltrc_yahoo/set1.train.txt","yahoo.train","yahoo.train.query","yahoo.train.group")
convert("Learning to Rank Challenge/ltrc_yahoo/set1.test.txt","yahoo.test","yahoo.test.query","yahoo.test.group")

In [16]:
# Load the LIBSVM file
X, y = load_svmlight_file('yahoo.train')


In [17]:
# Load the LIBSVM file
g = load_svmlight_file('yahoo.train.group')
groups = g[1]

In [18]:
# Replace the groups variable with this code to convert group sizes to group IDs
group_ids = [group_num for group_num, count in enumerate([int(x) for x in groups], start=1) for _ in range(count)]

In [19]:
max_groups = 400

In [20]:
n = len([x for x in group_ids if x <= max_groups])

In [21]:
X = X[:n].toarray()
y = y[:n]

In [22]:
group_ids = [x for x in group_ids if x <= max_groups]

In [23]:

X_2 = np.concatenate(
    (X, np.array(group_ids)[:, np.newaxis]),
    axis=1
)
y_2 = np.concatenate(
    (y[:, np.newaxis], np.array(group_ids)[:, np.newaxis]),
    axis=1
)

In [24]:
gkf = GroupKFold(n_splits=5)


param_grid = {
    'C': [0.01, 100],
    'max_iter': [1000, 5000],
    'tol': [1e-4],
    'penalty': ['l2']
}

Example Cross-Validating the Pairwise Model

In [25]:
results = []
scores = []

for penalty in param_grid['penalty']:
    for c in param_grid['C']:
        for max_iter in param_grid['max_iter']:
            for tol in param_grid['tol']:
                fold_scores = []
                for fold_idx, (train_idx, val_idx) in enumerate(gkf.split(X, y_2, group_ids)):

                    # Get training and validation DataFrames
                    X_train_fold = X_2[train_idx].copy()
                    y_train_fold = y_2[train_idx].copy()
                    X_val_fold = X_2[val_idx].copy()
                    y_val_fold = y_2[val_idx].copy()
                    
                    # Create DataFrame for easier handling with session_id
                    y_val_df_eval = pd.DataFrame({
                        'label': y_val_fold[:, 0],
                        'group': y_val_fold[:, 1]
                    })

                    # Fit and evaluate using calculate_ranking_ndcg
                    fold_model = RankSVM(C=c, tol=tol, max_iter=max_iter, penalty=penalty, dual=False, random_state=42)
                    fold_model.fit(X_train_fold, y_train_fold)
                    preds = fold_model.predict(X_val_fold)
                    y_val_df_eval['y_pred'] = preds
                    fold_score = calculate_ranking_ndcg(y_val_df_eval.label,
                                                       y_val_df_eval.y_pred,
                                                       get_group_sizes(y_val_df_eval))

                    fold_scores.append(round(fold_score,4))
                    print(f"Fold {fold_idx + 1} NDCG: {fold_score:.4f}")
                res = {'C': c, 'tol': tol, 'max_iter': max_iter, 'penalty':penalty, 'scores':fold_scores, 'mean_score':round(np.mean(fold_scores),4)}
                results.append(res)
                print ('folds complete', res)

Fold 1 NDCG: 0.8094
Fold 2 NDCG: 0.4878
Fold 3 NDCG: 0.9727
Fold 4 NDCG: 0.8350
Fold 5 NDCG: 0.8016
folds complete {'C': 0.01, 'tol': 0.0001, 'max_iter': 1000, 'penalty': 'l2', 'scores': [np.float64(0.8094), np.float64(0.4878), np.float64(0.9727), np.float64(0.835), np.float64(0.8016)], 'mean_score': np.float64(0.7813)}
Fold 1 NDCG: 0.8094
Fold 2 NDCG: 0.4878
Fold 3 NDCG: 0.9727
Fold 4 NDCG: 0.8350
Fold 5 NDCG: 0.8016
folds complete {'C': 0.01, 'tol': 0.0001, 'max_iter': 5000, 'penalty': 'l2', 'scores': [np.float64(0.8094), np.float64(0.4878), np.float64(0.9727), np.float64(0.835), np.float64(0.8016)], 'mean_score': np.float64(0.7813)}
Fold 1 NDCG: 0.8094
Fold 2 NDCG: 0.4878
Fold 3 NDCG: 0.9727
Fold 4 NDCG: 0.8350
Fold 5 NDCG: 0.8016
folds complete {'C': 100, 'tol': 0.0001, 'max_iter': 1000, 'penalty': 'l2', 'scores': [np.float64(0.8094), np.float64(0.4878), np.float64(0.9727), np.float64(0.835), np.float64(0.8016)], 'mean_score': np.float64(0.7813)}
Fold 1 NDCG: 0.8094
Fold 2 NDCG: 0.

In [ ]:
# catch warnings becuase I don't want to publish the full file path of my installations
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)

Compare to Pointwise Model

In [32]:
results = []
scores = []

for penalty in param_grid['penalty']:
    for c in param_grid['C']:
        for max_iter in param_grid['max_iter']:
            for tol in param_grid['tol']:
                fold_scores = []
                for fold_idx, (train_idx, val_idx) in enumerate(gkf.split(X, y_2, group_ids)):

                    # Get training and validation DataFrames
                    X_train_fold = X[train_idx].copy()
                    y_train_fold = y[train_idx].copy()
                    X_val_fold = X[val_idx].copy()
                    y_val_fold = y[val_idx].copy()
                    
                    # Create DataFrame for easier handling with grouping/query parameter
                    y_val_df_eval_pointwise = pd.DataFrame({
                        'label': y_2[val_idx, 0],
                        'group': y_2[val_idx, 1]
                    })

                    # Fit and evaluate using calculate_ranking_ndcg
                    fold_model = svm.LinearSVC(C=c, tol=tol, max_iter=max_iter, penalty=penalty, dual=False, random_state=42)
                    fold_model.fit(X_train_fold, y_train_fold)
                    y_val_df_eval_pointwise['y_pred'] = get_group_rankings(fold_model, X_val_fold, y_2[val_idx, 1])
                    fold_score = calculate_ranking_ndcg(y_val_df_eval_pointwise.label,
                                                       y_val_df_eval_pointwise.y_pred,
                                                       get_group_sizes(y_val_df_eval_pointwise))

                    fold_scores.append(round(fold_score,4))
                    print(f"Fold {fold_idx + 1} NDCG: {fold_score:.4f}")
                res = {'C': c, 'tol': tol, 'max_iter': max_iter, 'penalty':penalty, 'scores':fold_scores, 'mean_score':round(np.mean(fold_scores),4)}
                results.append(res)
                print ('folds complete', res)
prediction = fold_model.predict(X)

Fold 1 NDCG: 0.6530
Fold 2 NDCG: 0.6649
Fold 3 NDCG: 0.9822
Fold 4 NDCG: 0.8284
Fold 5 NDCG: 0.8832
folds complete {'C': 0.01, 'tol': 0.0001, 'max_iter': 1000, 'penalty': 'l2', 'scores': [np.float64(0.653), np.float64(0.6649), np.float64(0.9822), np.float64(0.8284), np.float64(0.8832)], 'mean_score': np.float64(0.8023)}
Fold 1 NDCG: 0.6530
Fold 2 NDCG: 0.6649
Fold 3 NDCG: 0.9822
Fold 4 NDCG: 0.8284
Fold 5 NDCG: 0.8832
folds complete {'C': 0.01, 'tol': 0.0001, 'max_iter': 5000, 'penalty': 'l2', 'scores': [np.float64(0.653), np.float64(0.6649), np.float64(0.9822), np.float64(0.8284), np.float64(0.8832)], 'mean_score': np.float64(0.8023)}
Fold 1 NDCG: 0.6606
Fold 2 NDCG: 0.7260
Fold 3 NDCG: 0.9822
Fold 4 NDCG: 0.8488
Fold 5 NDCG: 0.7189
folds complete {'C': 100, 'tol': 0.0001, 'max_iter': 1000, 'penalty': 'l2', 'scores': [np.float64(0.6606), np.float64(0.726), np.float64(0.9822), np.float64(0.8488), np.float64(0.7189)], 'mean_score': np.float64(0.7873)}
Fold 1 NDCG: 0.6606
Fold 2 NDCG: 0.